In [ ]:
# math
import math
import numpy as np
import pandas as pd

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# other
import json
import copy
import os

from experiments_2022 import DATASETS_PATH, IMAGE_PATH
from experiments_2022.zone_level_analysis import (
    base,
    cleaning,
    viz,
    regression_functions,
    clustering,
)
from experiments_2022.datasets import (
    load_zones,
    load_weather,
    pull_from_dataset,
)

from utils import (
    PROJECTS_2022,
    PROJECTS_2022_TOTAL,
    PROJECTS_VAV,
    PROJECTS_FC,
    NO_WEEKENDS,
    CONTROL_FOR_WEEKENDS,
    SUMMER_START_2022,
    SUMMER_END_2022,
    DOMINANT_THRESH,
    ROGUE_THRESH,
    REACTIVE_THRESH,
    RESPONSE_COLORS,
    DOMINANT_COLORS,
    SINGLE_PLOT_TXT_SIZE,
    SINGLE_PLOT_LEGEND_SIZE,
    SINGLE_PLOT_ANNOTATION_SIZE,
    MULTI_PLOT_TXT_SIZE,
    MULTI_PLOT_TITLE_SIZE,
    MULTI_PLOT_LEGEND_SIZE,
    MULTI_PLOT_ANNOTATION_SIZE,
    gini,
    add_line_to_subplots,
    run_equip_regressions,
    add_total_to_results_dict,
)

In [ ]:
if not os.path.exists("dicts_dfs"):
    os.makedirs("dicts_dfs")

In [ ]:
pd.set_option("display.max_rows", 1000000)

In [ ]:
def get_data(
    this_var,
    clean_this_var=None,
    projects=PROJECTS_2022,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=True,
    SI_units=False,
    remove_FCUs=False,
    clean_underyling_data=False,
):
    dfs = pull_from_dataset(
        "2022",
        projects,
        this_var,
        clean_data=clean_underyling_data,
        resample_rule="1h",
        resample_statistic="Mean",
    )
    dfs = cleaning.clean_dfs(
        dfs=dfs,
        this_var=clean_this_var,
        start_date=start_date,
        end_date=end_date,
        only_business_hours=only_business_hours,
        no_weekends=no_weekends,
        remove_FCUs=remove_FCUs,
    )
    if clean_this_var is None:
        clean_this_var = this_var

    zonal_schedules = {}
    for project in projects:
        zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
            project, experiment_year="2022", freq="hourly", df=dfs[project]
        )

    df_filter = cleaning.create_sp_filter(
        zonal_schedules, sps=[74], reverse_filter=False
    )
    dfs_control = cleaning.clean_dfs(
        dfs=dfs, only_business_hours=False, no_weekends=False, df_filter=df_filter
    )
    dfs_control_mean = base.run_passive_test_on_dfs(
        dfs=dfs_control, this_test="Mean", col_name="Mean"
    )
    return dfs, dfs_control, dfs_control_mean

In [ ]:
def get_lorenz_curves(dfs):
    all_lorenz = {}
    for project in dfs:
        this_df = dfs[project].iloc[:, 0].dropna()
        this_df = this_df.sort_values(ascending=False)
        lorenz = this_df.cumsum() / this_df.sum()
        all_lorenz[project] = lorenz.to_frame()
    return all_lorenz

In [ ]:
def get_dominant_zones(
    dfs,
    crs,
    dominant_thresh=DOMINANT_THRESH,
    rogue_thresh=ROGUE_THRESH,
    dominant_greater_lesser="lesser",
    to_csv=False,
):
    dominant_zones = {}
    for project in dfs:
        these_dzs = pd.Series(0, index=dfs[project].index)
        # dominant
        ser1 = dfs[project]
        if dominant_greater_lesser == "lesser":
            dominant = list((ser1[ser1 <= dominant_thresh]).dropna().index)
        elif dominant_greater_lesser == "greater":
            dominant = list((ser1[ser1 >= dominant_thresh]).dropna().index)
        these_dzs[dominant] = 1
        ser2 = crs[project]
        rogue = list((ser2[ser2 >= rogue_thresh]).dropna().index)
        rogue = list(set(rogue).intersection(set(list(these_dzs.index))))
        these_dzs[rogue] = 2
        these_dzs = these_dzs.to_frame()
        dominant_zones[project] = these_dzs
        if to_csv:
            dominant_zones[project].to_csv(f"./dicts_dfs/dominant/{project}.csv")
    return dominant_zones

In [ ]:
def add_gini(fig, lorenz, control_mean, annotation_size=MULTI_PLOT_ANNOTATION_SIZE):
    projects = list(lorenz.keys())
    gini_ser = pd.Series(index=projects)
    for project in projects:
        gini_ser[project] = gini(control_mean[project].iloc[:, 0].dropna())

    for i in range(len(projects)):
        project = projects[i]
        row = i // 3 + 1
        col = i % 3 + 1

        fig.add_annotation(
            text=f"Gini = {gini_ser[project]:.2f}",
            xref=f"x{i+1}",
            yref=f"y{i+1}",
            x=1,
            y=0,
            showarrow=False,
            xanchor="right",
            yanchor="bottom",
            font=dict(size=annotation_size, color="black"),
        )
    return fig

In [ ]:
def count_dzs(dominant_zones, total=False, vav_fc=False):
    projects = list(dominant_zones.keys())
    count = pd.DataFrame(
        index=projects, columns=["Rogue", "Dominant", "Dominated", "Total"]
    )
    for project in projects:
        count.loc[project, "Dominated"] = len(
            list(dominant_zones[project][dominant_zones[project] == 0].dropna().index)
        )
        count.loc[project, "Dominant"] = len(
            list(dominant_zones[project][dominant_zones[project] == 1].dropna().index)
        )
        count.loc[project, "Rogue"] = len(
            list(dominant_zones[project][dominant_zones[project] == 2].dropna().index)
        )
        count.loc[project, "Total"] = len(list(dominant_zones[project].index))
    if vav_fc:
        count.loc["AH-VAV", :] = count.loc[PROJECTS_VAV, :].sum(axis=0)
        count.loc["FC", :] = count.loc[PROJECTS_FC, :].sum(axis=0)
    if total:
        count.loc["TOTAL", :] = count.sum(axis=0)
    count_norm = pd.DataFrame(
        index=count.index, columns=["Rogue", "Dominant", "Dominated"]
    )
    count_norm["Rogue"] = count["Rogue"] / count["Total"]
    count_norm["Dominant"] = count["Dominant"] / count["Total"]
    count_norm["Dominated"] = count["Dominated"] / count["Total"]
    return count_norm

In [ ]:
def count_dz_crs(dominant_zones, crs, total=False, vav_fc=False):
    projects = list(dominant_zones.keys())
    count_CRs = pd.DataFrame(
        index=projects, columns=["Rogue", "Dominant", "Dominated", "Total"]
    )
    for project in projects:
        these_dzs = dominant_zones[project]
        common = list(
            set(list(these_dzs.index)).intersection(set(list(crs[project].index)))
        )
        these_dzs = these_dzs.loc[common, :]
        count_CRs.loc[project, "Dominated"] = (
            crs[project]
            .loc[list(these_dzs[these_dzs == 0].dropna().index), :]
            .sum()
            .iloc[0]
        )
        count_CRs.loc[project, "Dominant"] = (
            crs[project]
            .loc[list(these_dzs[these_dzs == 1].dropna().index), :]
            .sum()
            .iloc[0]
        )
        count_CRs.loc[project, "Rogue"] = (
            crs[project]
            .loc[list(these_dzs[these_dzs == 2].dropna().index), :]
            .sum()
            .iloc[0]
        )
        count_CRs.loc[project, "Total"] = crs[project].sum().sum()
        other = (
            count_CRs.loc[project, "Total"]
            - count_CRs.loc[project, "Dominated"]
            - count_CRs.loc[project, "Dominant"]
            - count_CRs.loc[project, "Rogue"]
        )
        if other > 0.001:
            count_CRs.loc[project, "Other"] = other
    if vav_fc:
        count_CRs.loc["AH-VAV", :] = count_CRs.loc[PROJECTS_VAV, :].sum(axis=0)
        count_CRs.loc["FC", :] = count_CRs.loc[PROJECTS_FC, :].sum(axis=0)
    if total:
        count_CRs.loc["TOTAL", :] = count_CRs.sum(axis=0)
    count_CRs_norm = pd.DataFrame(
        index=count_CRs.index, columns=["Rogue", "Dominant", "Dominated"]
    )
    count_CRs_norm["Rogue"] = count_CRs["Rogue"] / count_CRs["Total"]
    count_CRs_norm["Dominant"] = count_CRs["Dominant"] / count_CRs["Total"]
    count_CRs_norm["Dominated"] = count_CRs["Dominated"] / count_CRs["Total"]
    if "Other" in count_CRs.columns:
        count_CRs_norm["Other"] = count_CRs["Other"] / count_CRs["Total"]
    return count_CRs_norm

In [ ]:
def quantify_heatmap_dfs(dominant_zones, reactive_zones):
    heat_maps_row = {}
    heat_maps_col = {}

    for project in dominant_zones:
        heat_map = pd.DataFrame(
            0,
            index=[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
                "Small change zonal load (remained high)",
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
                "Small change zonal load + Typically in heating",
            ],
            columns=["Rogue", "Dominant", "Dominated"],
        )

        # grab lists
        rzs = reactive_zones[project].iloc[:, 0]
        dzs = dominant_zones[project].iloc[:, 0]
        common = list(set(list(rzs.index)).intersection(set(list(dzs.index))))
        rzs = rzs[common]
        dzs = dzs[common]

        green = list(rzs[rzs == 0].index)  # green
        orange = list(rzs[rzs == 2].index)  # orange
        red = list(rzs[rzs == 3].index)  # red
        blue_grey = list(rzs[rzs == 1].index)  # blue
        blue_grey.extend(list(rzs[rzs == 4].index))  # grey

        dominant = list(dzs[dzs == 1].index)
        rogue = list(dzs[dzs == 2].index)
        dominated = list(dzs[dzs == 0].index)

        heat_map.loc[
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more", "Rogue"
        ] = len(set(green).intersection(set(rogue)))
        heat_map.loc[
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more", "Dominant"
        ] = len(set(green).intersection(set(dominant)))
        heat_map.loc[
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more", "Dominated"
        ] = len(set(green).intersection(set(dominated)))

        heat_map.loc["Small change zonal load (remained high)", "Rogue"] = len(
            set(orange).intersection(set(rogue))
        )
        heat_map.loc["Small change zonal load (remained high)", "Dominant"] = len(
            set(orange).intersection(set(dominant))
        )
        heat_map.loc["Small change zonal load (remained high)", "Dominated"] = len(
            set(orange).intersection(set(dominated))
        )

        heat_map.loc[
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more", "Rogue"
        ] = len(set(red).intersection(set(rogue)))
        heat_map.loc[
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more", "Dominant"
        ] = len(set(red).intersection(set(dominant)))
        heat_map.loc[
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more", "Dominated"
        ] = len(set(red).intersection(set(dominated)))

        heat_map.loc["Small change zonal load + Typically in heating", "Rogue"] = len(
            set(blue_grey).intersection(set(rogue))
        )
        heat_map.loc[
            "Small change zonal load + Typically in heating", "Dominant"
        ] = len(set(blue_grey).intersection(set(dominant)))
        heat_map.loc[
            "Small change zonal load + Typically in heating", "Dominated"
        ] = len(set(blue_grey).intersection(set(dominated)))

        # row based
        heat_map_norm = copy.deepcopy(heat_map)
        heat_map_norm.loc[
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more", :
        ] /= len(green)
        heat_map_norm.loc["Small change zonal load (remained high)", :] /= len(orange)
        heat_map_norm.loc[
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more", :
        ] /= len(red)
        heat_map_norm.loc["Small change zonal load + Typically in heating", :] /= len(
            blue_grey
        )
        heat_maps_row[project] = heat_map_norm

        # col based
        heat_map_norm = copy.deepcopy(heat_map)
        heat_map_norm.loc[:, "Rogue"] /= len(rogue)
        heat_map_norm.loc[:, "Dominant"] /= len(dominant)
        heat_map_norm.loc[:, "Dominated"] /= len(dominated)
        heat_maps_col[project] = heat_map_norm

    return heat_maps_row, heat_maps_col

In [ ]:
def plot_confusion_heatmaps(
    y_data,
    title="Title",
    text_size=viz.TXT_SZ,
    num_cols=3,
    width=800,
    height=500,
    horizontal_spacing=0.1,
    vertical_spacing=0.1,
    show_colorbar=False,
    colorbar_min=0,
    colorbar_max=1,
    min_color="white",
    max_color="green",
    color_bar_title="",
    color_bar_side="right",
):
    """
    Plot a grid of heatmaps (one per building) for confusion matrices.

    Parameters
    ----------
    y_data : dict[str, pandas.DataFrame]
        Mapping of building name -> heat map data (df)
    title : str
    text_size : int
    num_cols : int | None
    width : int
    height : int
    horizontal_spacing : float
    vertical_spacing : float
    show_colorbar : bool
    colorbar_min : float
    colorbar_max : float
    min_color : str
    max_color : str
    color_bar_title : str

    Returns
    -------
    plotly.graph_objects.Figure
    """
    if not isinstance(y_data, dict):
        y_data = {title: y_data}

    # subplot titles
    these_keys = list(y_data.keys())
    if these_keys == [title]:
        subplot_titles = None  # edge case, we entered a df with no title
    else:
        subplot_titles = these_keys

    # create plot
    if len(these_keys) >= num_cols:
        graph_num_cols = num_cols
    else:
        graph_num_cols = len(these_keys)
    graph_num_rows = math.ceil((len(these_keys)) / graph_num_cols)

    fig = make_subplots(
        rows=graph_num_rows,
        cols=graph_num_cols,
        subplot_titles=subplot_titles,
        horizontal_spacing=horizontal_spacing,
        vertical_spacing=vertical_spacing,
    )

    # Shared color axis (so one colorbar can represent all)
    fig.update_layout(
        coloraxis=dict(
            colorscale=[[colorbar_min, min_color], [colorbar_max, max_color]],
            cmin=colorbar_min,
            cmax=colorbar_max,
            colorbar=dict(
                title=dict(
                    text=color_bar_title,
                    side=color_bar_side,
                ),
                x=1.02,  # slightly outside the right edge
                len=0.8,
                tickfont=dict(size=text_size),
            ),
        ),
        coloraxis_showscale=show_colorbar,
        title=title,
        width=width * graph_num_cols,
        height=height * graph_num_rows,
    )

    # Add one heatmap per building
    idx = 1
    for this_key in y_data:
        df = y_data[this_key]
        z = df.values
        x = list(df.columns)
        y = list(df.index)

        # Row/col in subplot grid
        row = (idx - 1) // num_cols + 1
        col = (idx - 1) % num_cols + 1

        # Add heatmap trace using the shared coloraxis
        fig.add_trace(
            go.Heatmap(
                z=z,
                x=x,
                y=y,
                coloraxis="coloraxis",
                showscale=False,  # colorbar handled globally by coloraxis
            ),
            row=row,
            col=col,
        )

        # Centered percentage annotations for this subplot
        # Need to target the specific subplot's axes via xref/yref
        axis_num = (row - 1) * num_cols + col  # subplot index → axis suffix
        for i, yi in enumerate(y):
            for j, xj in enumerate(x):
                fig.add_annotation(
                    x=xj,
                    y=yi,
                    text=f"{z[i, j]:.1%}",
                    showarrow=False,
                    font=dict(size=text_size, color="black"),
                    xref=f"x{axis_num}",
                    yref=f"y{axis_num}",
                )
        idx += 1

    # Global axis styling for all subplots
    fig.update_xaxes(side="top", tickfont=dict(size=text_size))
    fig.update_yaxes(autorange="reversed", tickfont=dict(size=text_size))

    fig.update_layout(title=title, title_font=dict(size=text_size), title_x=0)
    return fig

# Define broader groups

In [ ]:
ez_csv = pd.read_csv(DATASETS_PATH / "csvs/2022_experiment_csvs/excluded_zones.csv")
ALL_ZONES = {}
EXCLUDED_ZONES = {}
ZONES_BY_TYPE = {}
for project in PROJECTS_2022:
    ZONES_BY_TYPE[project] = {}

    tload_zones = list(
        cleaning.clean_df(
            load_zones("2022", project, "zone-tloads"),
            this_var="zone-dummy",
            no_weekends=False,
            only_business_hours=False,
        ).columns
    )
    temp_zones = list(
        cleaning.clean_df(
            load_zones("2022", project, "zone-temps"),
            this_var="zone-dummy",
            no_weekends=False,
            only_business_hours=False,
        ).columns
    )
    csp_zones = list(
        cleaning.clean_df(
            load_zones("2022", project, "zone-coolsp"),
            this_var="zone-dummy",
            no_weekends=False,
            only_business_hours=False,
        ).columns
    )
    all_zones = list(set(tload_zones).union(set(temp_zones)).union(set(csp_zones)))

    vavs = [zone for zone in all_zones if "VAV" in zone.upper()]
    fcus = [
        zone for zone in all_zones if ("FC" in zone.upper()) or ("VVS" in zone.upper())
    ]
    all_zones = list(set(vavs).union(set(fcus)))

    ZONES_BY_TYPE[project]["VAVs"] = vavs
    ZONES_BY_TYPE[project]["FCUs"] = fcus
    ALL_ZONES[project] = all_zones

    ezs = list(ez_csv[project].dropna())
    ezs = list(set(ezs).intersection(set(all_zones)))
    ez_df = pd.DataFrame(0, index=all_zones, columns=["Excluded Zones"])
    ez_df.loc[ezs, :] = 1
    EXCLUDED_ZONES[project] = ez_df
    ez_df.to_csv(f"./dicts_dfs/excluded/{project}.csv")

with open("./dicts_dfs/all_zones.json", "w") as f:
    json.dump(ALL_ZONES, f, indent=4)

with open("./dicts_dfs/zones_by_type.json", "w") as f:
    json.dump(ZONES_BY_TYPE, f, indent=4)

In [ ]:
AHUs = {}
for project in PROJECTS_2022:
    dat = load_zones("2022", project, "ahu-dat")
    AHUs[project] = list(cleaning.clean_df(dat, "ahu-dat").columns)

with open("./dicts_dfs/ahus.json", "w") as f:
    json.dump(AHUs, f, indent=4)

In [ ]:
EXCLUDED_ZONES_TOT = add_total_to_results_dict(EXCLUDED_ZONES)

# Dominant zones

In [ ]:
(crs, crs_control, CRs_CONTROL_MEAN) = get_data(
    this_var="zone-simple_cooling_requests",
    projects=PROJECTS_2022,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=True,
    clean_underyling_data=True,
    clean_this_var="zone-dummy",
)
CRs_CONTROL_MEAN_TOT = add_total_to_results_dict(CRs_CONTROL_MEAN)

In [ ]:
LORENZ_ALL = get_lorenz_curves(CRs_CONTROL_MEAN)
LORENZ_ALL_TOT = get_lorenz_curves(CRs_CONTROL_MEAN_TOT)

In [ ]:
DOMINANT_ZONES = get_dominant_zones(
    LORENZ_ALL,
    CRs_CONTROL_MEAN,
    dominant_thresh=DOMINANT_THRESH,
    rogue_thresh=ROGUE_THRESH,
    dominant_greater_lesser="lesser",
    to_csv=True,
)
DOMINANT_ZONES_TOT = add_total_to_results_dict(DOMINANT_ZONES)

## Count zones

In [ ]:
count_norm = count_dzs(DOMINANT_ZONES_TOT, total=False, vav_fc=True)

In [ ]:
100 * count_norm

In [ ]:
100 * (count_norm["Rogue"] + count_norm["Dominant"])

In [ ]:
fig = viz.make_bar_plot(
    y_data=count_norm.loc[PROJECTS_2022_TOTAL, :],
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        }
    },
    y_axis_title="Fraction of Zones [Unitless]",
    y_range=[0, 1],
    annotations=[
        "Dominated",
        "Dominant",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022_TOTAL))],
    width=1200,
    height=600,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    xaxis=dict(range=[-0.25, len(PROJECTS_2022_TOTAL)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC1.png")

## CRs

In [ ]:
count_CRs_norm = count_dz_crs(
    DOMINANT_ZONES_TOT, CRs_CONTROL_MEAN_TOT, total=False, vav_fc=True
)

In [ ]:
100 * count_CRs_norm

In [ ]:
100 * (count_CRs_norm["Rogue"] + count_CRs_norm["Dominant"])

In [ ]:
fig = viz.make_bar_plot(
    y_data=count_CRs_norm.loc[PROJECTS_2022_TOTAL, :],
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        }
    },
    y_axis_title="Fraction of Cooling Requests<br>Control Days [Unitless]",
    y_range=[0, 1],
    annotations=[
        "Dominated",
        "Dominant",
        "Rogue",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022_TOTAL))],
    width=1200,
    height=600,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    xaxis=dict(range=[-0.25, len(PROJECTS_2022_TOTAL)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC2.png")

# Reactive zones

## Delta tload

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    only_business_hours=True,
    no_weekends=False,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
)["temperature"]

In [ ]:
(TLOADS, tloads_control, TLOADS_CONTROL_MEAN) = get_data(
    this_var="zone-tloads",
    clean_this_var="zone-tloads",
    projects=PROJECTS_2022,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=True,
)

In [ ]:
TLOADS_CONTROL_MEAN_TOT = add_total_to_results_dict(TLOADS_CONTROL_MEAN)

In [ ]:
(
    deltas_76_tloads_2022,
    deltas_low_76_tloads_2022,
    deltas_high_76_tloads_2022,
    deltas_78_tloads,
    deltas_low_78_tloads,
    deltas_high_78_tloads,
) = run_equip_regressions(
    TLOADS,
    T,
    "Absolute Change",
)

In [ ]:
REACTIVE_ZONES = clustering.run_1D_clustering_on_dict(
    deltas_78_tloads,
    slices=[REACTIVE_THRESH["Pos"], REACTIVE_THRESH["Neg"]],
    mapping={2: 0, 1: 1, 0: 3},
)

## Corrections based on control days

In [ ]:
for project in PROJECTS_2022:
    # small change in tload, but remained high
    these_rzs = REACTIVE_ZONES[project].iloc[:, 0]
    these_rzs = list(these_rzs[these_rzs == 1].index)
    these_control = TLOADS_CONTROL_MEAN[project].iloc[:, 0]
    these_control = list(
        these_control[these_control >= REACTIVE_THRESH["High Constant"]].index
    )
    correct = list(set(these_rzs).intersection(set(these_control)))
    REACTIVE_ZONES[project].loc[correct, :] = 2

for project in PROJECTS_2022:
    # in heating
    these_control = TLOADS_CONTROL_MEAN[project].iloc[:, 0]
    these_control = list(
        these_control[these_control <= REACTIVE_THRESH["Heating"]].index
    )
    REACTIVE_ZONES[project].loc[these_control, :] = 4

for project in PROJECTS_2022:
    REACTIVE_ZONES[project] = REACTIVE_ZONES[project].dropna()
    REACTIVE_ZONES[project].to_csv(f"./dicts_dfs/reactive/{project}.csv")

In [ ]:
REACTIVE_ZONES_TOT = add_total_to_results_dict(REACTIVE_ZONES)

## Detailed plot

In [ ]:
tloads_control_std = base.run_passive_test_on_dfs(
    dfs=tloads_control, this_test="Std", col_name="Std"
)

In [ ]:
(
    this_deltas_76_tloads_2022,
    this_deltas_high_76_tloads_2022,
    this_deltas_low_76_tloads_2022,
    this_deltas_78_tloads,
    this_deltas_high_78_tloads,
    this_deltas_low_78_tloads,
    this_control_tloads,
    # this_tloads_control_std,
) = base.make_common_index(
    [
        copy.deepcopy(deltas_76_tloads_2022),
        copy.deepcopy(deltas_low_76_tloads_2022),
        copy.deepcopy(deltas_high_76_tloads_2022),
        copy.deepcopy(deltas_78_tloads),
        copy.deepcopy(deltas_high_78_tloads),
        copy.deepcopy(deltas_low_78_tloads),
        copy.deepcopy(TLOADS_CONTROL_MEAN),
        # copy.deepcopy(tloads_control_std),
    ]
)

In [ ]:
this_deltas_78_tloads_tot = add_total_to_results_dict(this_deltas_78_tloads)
this_deltas_high_78_tloads_tot = add_total_to_results_dict(this_deltas_high_78_tloads)
this_deltas_low_78_tloads_tot = add_total_to_results_dict(this_deltas_low_78_tloads)
this_control_tloads_tot = add_total_to_results_dict(this_control_tloads)
# this_control_tloads_std_tot =  add_total_to_results_dict(this_tloads_control_std)

In [ ]:
fig = viz.make_scatter_plot(
    y_data=this_deltas_78_tloads_tot,
    y_error_up_data=this_deltas_high_78_tloads_tot,
    y_error_down_data=this_deltas_low_78_tloads_tot,
    y_axis_title="Absolute Change in<br>Zonal Load [%]",
    x_data=this_control_tloads_tot,
    # x_error_right_data=this_control_tloads_std_tot,
    # x_error_left_data=this_control_tloads_std_tot,
    x_axis_title="Zonal Load [%]<br>Control Days",
    color_data=REACTIVE_ZONES_TOT,
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            4: "Typically in heating",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            4: RESPONSE_COLORS["Typically in heating"],
        },
    },
    shape_data=EXCLUDED_ZONES_TOT,
    shape_legend={
        "name": {
            0: "Included",
            1: "Excluded",
        },
        "shape": {
            0: "circle",
            1: "x",
        },
    },
    num_cols=3,
    horizontal_spacing=0.125,
    vertical_spacing=0.125,
    height=500,
    width=800,
    y_range=[-110, 110],
    x_range=[-110, 110],
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.075,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)
p = 0
for i in range(1, 4 + 1):  # rows
    for j in range(1, 3 + 1):  # cols
        if i == 4 and j > 2:
            continue
        project = list(this_deltas_78_tloads_tot.keys())[p]
        for slice in [REACTIVE_THRESH["Pos"], REACTIVE_THRESH["Neg"]]:
            fig.add_shape(
                type="line",
                x0=-110,
                x1=110,
                y0=slice,
                y1=slice,
                line=dict(color="Black", width=3, dash="solid"),
                row=i,
                col=j,
            )
            fig.add_shape(
                type="line",
                x0=REACTIVE_THRESH["Heating"],
                x1=REACTIVE_THRESH["Heating"],
                y0=-110,
                y1=110,
                line=dict(color="Black", width=3, dash="solid"),
                row=i,
                col=j,
            )
        fig.add_shape(
            type="line",
            x0=REACTIVE_THRESH["High Constant"],
            x1=REACTIVE_THRESH["High Constant"],
            y0=REACTIVE_THRESH["Pos"],
            y1=REACTIVE_THRESH["Neg"],
            line=dict(color="Black", width=3, dash="solid"),
            row=i,
            col=j,
        )
        p += 1

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureA2.png")

### Check regression

In [ ]:
project = "OFF-2"
zone = "VAV 3-184"

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    only_business_hours=True,
    no_weekends=False,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
)["temperature"]

In [ ]:
df = TLOADS[project][zone].to_frame()

binary_df = regression_functions.get_2021_2022_binary_df(
    project=project,
    experiment_year="2022",
    freq="daily",
    baseline_column="CSP = 74F",
    drop_baseline_column=True,
    no_weekends=NO_WEEKENDS[project],
    control_for_weekends=CONTROL_FOR_WEEKENDS[project],
    zone=zone,
)

reg_results = regression_functions.general_Delta_fn(
    df=df,
    T=T,
    binary=binary_df,
    mode="Absolute Change",
    summary_statistic="Mean",
)

In [ ]:
fig = viz.plot_experiment_regression(
    experiment_results=reg_results,
    df=df,
    T=T,
    binary=binary_df,
    line_legend={
        "name": {
            "Control": "CSP = 23.3C",
            "CSP = 76F": "CSP = 24.4C",
            "CSP = 78F": "CSP = 25.5C",
        },
        "color": {
            "Control": "RoyalBlue",
            "CSP = 76F": "DarkOrange",
            "CSP = 78F": "Firebrick",
        },
    },
    mode="Absolute Change",
    summary_statistic="Mean",
    line_width=2.5,
    y_axis_title="Daily Average<br>Zonal Load (%)",
    x_axis_title="Average Daytime OAT (C)",
    height=550,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.225,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

new_titles = ["Zonal Level"]
for i, annotation in enumerate(fig.layout.annotations):
    annotation.text = new_titles[i]

In [ ]:
# fig

## Count

In [ ]:
categories = [
    f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
    "Small change zonal load",
    "Small change zonal load (remained high)",
    f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
    "Typically in heating",
]

excluded_labels = ["included", "excluded"]

labels = []
for ex in excluded_labels:
    for cat in categories:
        labels.append(f"{cat} ({ex})")

projects_total = copy.deepcopy(PROJECTS_2022_TOTAL)
projects_total.extend(["OFF", "LAB"])

tot_off_zones = 0
tot_lab_zones = 0
tot_zones = 0

rz_summary = pd.DataFrame(0, index=projects_total, columns=labels)

In [ ]:
for project in PROJECTS_2022_TOTAL:
    these_reactive_zones = REACTIVE_ZONES_TOT[project].iloc[:, 0]
    these_excluded_zones = EXCLUDED_ZONES_TOT[project].iloc[:, 0]
    common = list(
        set(list(these_reactive_zones.index)).intersection(
            set(list(these_excluded_zones.index))
        )
    )
    these_reactive_zones = these_reactive_zones[common]
    these_excluded_zones = these_excluded_zones[common]
    for i in range(len(categories)):
        group = list((these_reactive_zones[these_reactive_zones == i]).index)
        for j in range(len(excluded_labels)):
            criticality = list((these_excluded_zones[these_excluded_zones == j]).index)
            these_zones = list(set(group).intersection(set(criticality)))
            rz_summary.loc[project, f"{categories[i]} ({excluded_labels[j]})"] = len(
                these_zones
            ) / len(these_reactive_zones)
            if "OFF" in project:
                rz_summary.loc["OFF", f"{categories[i]} ({excluded_labels[j]})"] += len(
                    these_zones
                )
            else:
                rz_summary.loc["LAB", f"{categories[i]} ({excluded_labels[j]})"] += len(
                    these_zones
                )
    if "OFF" in project:
        tot_off_zones += len(these_reactive_zones)
    else:
        tot_lab_zones += len(these_reactive_zones)

In [ ]:
rz_summary.loc["OFF", :] = rz_summary.loc["OFF", :] / tot_off_zones
rz_summary.loc["LAB", :] = rz_summary.loc["LAB", :] / tot_lab_zones

In [ ]:
100 * rz_summary

In [ ]:
fig = viz.make_bar_plot(
    y_data=rz_summary.loc[PROJECTS_2022_TOTAL, :],
    bar_legend={
        "color": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            "Small change zonal load (included)": RESPONSE_COLORS[
                "Small change zonal load"
            ],
            "Small change zonal load (remained high) (included)": RESPONSE_COLORS[
                "Small change zonal load (remained high)"
            ],
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            "Typically in heating (included)": RESPONSE_COLORS["Typically in heating"],
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            "Small change zonal load (excluded)": RESPONSE_COLORS[
                "Small change zonal load"
            ],
            "Small change zonal load (remained high) (excluded)": RESPONSE_COLORS[
                "Small change zonal load (remained high)"
            ],
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            "Typically in heating (excluded)": RESPONSE_COLORS["Typically in heating"],
        },
        "name": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            "Small change zonal load (included)": "Small change zonal load",
            "Small change zonal load (remained high) (included)": "Small change zonal load (remained high)",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            "Typically in heating (included)": "Typically in heating",
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)",
            "Small change zonal load (excluded)": "Small change zonal load (excluded)",
            "Small change zonal load (remained high) (excluded)": "Small change zonal load (remained high) (excluded)",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)",
            "Typically in heating (excluded)": "Typically in heating (excluded)",
        },
        "opacity": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": 1,
            "Small change zonal load (included)": 1,
            "Small change zonal load (remained high) (included)": 1,
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": 1,
            "Typically in heating (included)": 1,
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": 0.75,
            "Small change zonal load (excluded)": 0.75,
            "Small change zonal load (remained high) (excluded)": 0.75,
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": 0.75,
            "Typically in heating (excluded)": 0.75,
        },
        "pattern": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": "",
            "Small change zonal load (included)": "",
            "Small change zonal load (remained high) (included)": "",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": "",
            "Typically in heating (included)": "",
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": "/",
            "Small change zonal load (excluded)": "/",
            "Small change zonal load (remained high) (excluded)": "/",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": "/",
            "Typically in heating (excluded)": "/",
        },
    },
    pattern_legend={"Excluded": ("/", 0.5)},
    dont_add_to_legend=[
        f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)",
        "Small change zonal load (excluded)",
        "Small change zonal load (remained high) (excluded)",
        f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)",
        "Typically in heating (excluded)",
    ],
    y_axis_title="Fraction of Zones [Unitless]",
    y_range=[0, 1],
    annotations=[
        f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)",
        "Small change zonal load (included)",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022_TOTAL))],
    width=1250,  # exception
    height=650,  # exception
    legend_size=SINGLE_PLOT_TXT_SIZE,  # exception
    text_size=SINGLE_PLOT_TXT_SIZE,
    annotation_size=SINGLE_PLOT_ANNOTATION_SIZE - 2,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    xaxis=dict(range=[-0.25, len(PROJECTS_2022_TOTAL)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure4B.png")

### Abstract

In [ ]:
fig = viz.make_bar_plot(
    y_data=rz_summary.loc[PROJECTS_2022_TOTAL, :],
    bar_legend={
        "color": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            "Small change zonal load (included)": RESPONSE_COLORS[
                "Small change zonal load"
            ],
            "Small change zonal load (remained high) (included)": RESPONSE_COLORS[
                "Small change zonal load (remained high)"
            ],
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            "Typically in heating (included)": RESPONSE_COLORS["Typically in heating"],
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            "Small change zonal load (excluded)": RESPONSE_COLORS[
                "Small change zonal load"
            ],
            "Small change zonal load (remained high) (excluded)": RESPONSE_COLORS[
                "Small change zonal load (remained high)"
            ],
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            "Typically in heating (excluded)": RESPONSE_COLORS["Typically in heating"],
        },
        "name": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            "Small change zonal load (included)": "Small change zonal load",
            "Small change zonal load (remained high) (included)": "Small change zonal load (remained high)",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            "Typically in heating (included)": "Typically in heating",
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)",
            "Small change zonal load (excluded)": "Small change zonal load (excluded)",
            "Small change zonal load (remained high) (excluded)": "Small change zonal load (remained high) (excluded)",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)",
            "Typically in heating (excluded)": "Typically in heating (excluded)",
        },
        "opacity": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": 1,
            "Small change zonal load (included)": 1,
            "Small change zonal load (remained high) (included)": 1,
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": 1,
            "Typically in heating (included)": 1,
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": 0.75,
            "Small change zonal load (excluded)": 0.75,
            "Small change zonal load (remained high) (excluded)": 0.75,
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": 0.75,
            "Typically in heating (excluded)": 0.75,
        },
        "pattern": {
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)": "",
            "Small change zonal load (included)": "",
            "Small change zonal load (remained high) (included)": "",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (included)": "",
            "Typically in heating (included)": "",
            f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)": "/",
            "Small change zonal load (excluded)": "/",
            "Small change zonal load (remained high) (excluded)": "/",
            f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)": "/",
            "Typically in heating (excluded)": "/",
        },
    },
    pattern_legend={"Excluded": ("/", 0.5)},
    dont_add_to_legend=[
        f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)",
        "Small change zonal load (excluded)",
        "Small change zonal load (remained high) (excluded)",
        f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more (excluded)",
        "Typically in heating (excluded)",
    ],
    y_axis_title="Fraction of Zones<br>[Unitless]",
    y_range=[0, 1],
    annotations=[
        f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)",
        "Small change zonal load (included)",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022_TOTAL))],
    width=1700,  # exception
    height=550,  # exception
    legend_size=MULTI_PLOT_TXT_SIZE + 10,  # exception
    text_size=MULTI_PLOT_TXT_SIZE + 6,
    annotation_size=MULTI_PLOT_ANNOTATION_SIZE,
)
fig = fig.update_layout(
    xaxis=dict(range=[-0.25, len(PROJECTS_2022_TOTAL)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/GraphAbstractBottomLeft.png")

### For presentation

In [ ]:
rz_summary_simple = pd.Series(
    index=[
        f"Reduced cooling load {abs(REACTIVE_THRESH['Neg'])}% or more",
        "Small change in cooling load",
        "Small change in cooling load (remained high)",
        "Other",
    ]
)
rz_summary_simple[f"Reduced cooling load {abs(REACTIVE_THRESH['Neg'])}% or more"] = (
    rz_summary.loc[
        "TOTAL", f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (included)"
    ]
    + rz_summary.loc[
        "TOTAL", f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more (excluded)"
    ]
)
rz_summary_simple["Small change in cooling load"] = (
    rz_summary.loc["TOTAL", "Small change zonal load (included)"]
    + rz_summary.loc["TOTAL", "Small change zonal load (excluded)"]
)
rz_summary_simple["Small change in cooling load (remained high)"] = (
    rz_summary.loc["TOTAL", "Small change zonal load (remained high) (included)"]
    + rz_summary.loc["TOTAL", "Small change zonal load (remained high) (excluded)"]
)
rz_summary_simple["Other"] = 1 - rz_summary_simple.sum()
rz_summary_simple = rz_summary_simple.to_frame().T
rz_summary_simple.index = [" "]

In [ ]:
fig = viz.make_bar_plot(
    y_data=rz_summary_simple,
    bar_legend={
        "color": {
            f"Reduced cooling load {abs(REACTIVE_THRESH['Neg'])}% or more": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            "Small change in cooling load": RESPONSE_COLORS["Small change zonal load"],
            "Small change in cooling load (remained high)": RESPONSE_COLORS[
                "Small change zonal load (remained high)"
            ],
            "Other": "Gray",
        },
    },
    y_axis_title="Fraction of Zones<br>[Unitless]",
    y_range=[0, 1],
    annotations=[
        f"Reduced cooling load {abs(REACTIVE_THRESH['Neg'])}% or more",
        "Small change in cooling load",
        "Excluded from experiments",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="group",
    width=700,  # exception
    height=500,  # exception
    legend_size=SINGLE_PLOT_TXT_SIZE,  # exception
    text_size=SINGLE_PLOT_TXT_SIZE,
    annotation_size=SINGLE_PLOT_ANNOTATION_SIZE - 2,
    legend_order="forward",
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
)

In [ ]:
# fig

# Compare reactive and dominant


## Lorenz curve

In [ ]:
fig = viz.make_dot_plot(
    LORENZ_ALL_TOT,
    y_range=[-0.3, 1.1],
    normalize_x=True,
    y_axis_title="Fraction of Cumulative<br>Cooling Requests<br>Control Days [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.1,
    horizontal_spacing=0.125,
    color_data=REACTIVE_ZONES_TOT,
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            4: "Typically in heating",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            4: RESPONSE_COLORS["Typically in heating"],
        },
    },
    y_zerolinecolor="LightGray",
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)
fig = add_gini(fig, LORENZ_ALL_TOT, CRs_CONTROL_MEAN_TOT)

draw_dominant_y = {}
draw_dominant_x = {}
draw_rogue_y = {}
draw_rogue_x = {}

# detemrine lines
for project in LORENZ_ALL_TOT:
    if project == "TOTAL":
        continue
    # rogue
    these_rogue = list(
        DOMINANT_ZONES_TOT[project][DOMINANT_ZONES_TOT[project] == 2].dropna().index
    )
    this_lorenz_rogue = LORENZ_ALL_TOT[project].loc[these_rogue, :].iloc[:, 0]
    this_lorenz_rogue = this_lorenz_rogue.sort_values(ascending=False)
    try:
        this_rogue = float(this_lorenz_rogue.iloc[0])
    except:
        this_rogue = np.nan
    draw_rogue_y[project] = this_rogue
    draw_rogue_x[project] = (len(this_lorenz_rogue) - 1) / len(LORENZ_ALL_TOT[project])

    # dominant
    these_dominant = list(
        DOMINANT_ZONES_TOT[project][DOMINANT_ZONES_TOT[project] == 1].dropna().index
    )
    this_lorenz_dominant = LORENZ_ALL_TOT[project].loc[these_dominant, :].iloc[:, 0]
    this_lorenz_dominant = this_lorenz_dominant.sort_values(ascending=False)
    try:
        this_dominant = float(this_lorenz_dominant.iloc[0])
    except:
        this_dominant = np.nan
    draw_dominant_y[project] = this_dominant
    draw_dominant_x[project] = (
        len(this_lorenz_dominant) + len(this_lorenz_rogue) - 1
    ) / len(LORENZ_ALL_TOT[project])

for i in range(1, len(PROJECTS_2022) + 1):
    project = PROJECTS_2022[i - 1]
    xref = "x" if i == 1 else f"x{i}"
    yref = "y" if i == 1 else f"y{i}"

    if not np.isnan(draw_rogue_y[project]):
        # rogue
        fig.add_shape(
            type="line",
            x0=-0.1,
            x1=draw_rogue_x[project],
            y0=draw_rogue_y[project],
            y1=draw_rogue_y[project],
            line=dict(color="black", dash="dash", width=3),
            xref=xref,
            yref=yref,
            layer="above",
        )
        fig.add_shape(
            type="line",
            x0=draw_rogue_x[project],
            x1=draw_rogue_x[project],
            y0=-0.3,
            y1=draw_rogue_y[project],
            line=dict(color="black", dash="dash", width=3),
            xref=xref,
            yref=yref,
            layer="above",
        )
        fig.add_annotation(
            text="Rogue",
            x=draw_rogue_x[project],
            y=draw_rogue_y[project],
            xanchor="left",
            yanchor="top",
            showarrow=False,
            font=dict(size=MULTI_PLOT_ANNOTATION_SIZE, color="black"),
            xref=xref,
            yref=yref,
            textangle=90,
        )
    if not np.isnan(draw_dominant_y[project]):
        fig.add_shape(
            type="line",
            x0=-0.1,
            x1=draw_dominant_x[project],
            y0=draw_dominant_y[project],
            y1=draw_dominant_y[project],
            line=dict(color="black", dash="dash", width=3),
            xref=xref,
            yref=yref,
            layer="above",
        )
        fig.add_shape(
            type="line",
            x0=draw_dominant_x[project],
            x1=draw_dominant_x[project],
            y0=-0.3,
            y1=draw_dominant_y[project],
            line=dict(color="black", dash="dash", width=3),
            xref=xref,
            yref=yref,
            layer="above",
        )
        fig.add_annotation(
            text="Dominant",
            x=draw_dominant_x[project],
            y=draw_dominant_y[project],
            xanchor="left",
            yanchor="top",
            showarrow=False,
            font=dict(size=MULTI_PLOT_ANNOTATION_SIZE, color="black"),
            xref=xref,
            yref=yref,
            textangle=90,
        )

fig = fig.update_yaxes(
    tickmode="array",
    tickvals=[-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1.0],
    ticktext=["", "0", "0.2", "0.4", "0.6", "0.8", "1.0"],
    matches="y",  # link all y-axes
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure9.png")

### Graphical abstract

In [ ]:
project = "OFF-5"

fig = viz.make_dot_plot(
    LORENZ_ALL_TOT[project],
    y_range=[-0.1, 1.1],
    normalize_x=True,
    y_axis_title="Fraction of Cumulative<br>Cooling Requests [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.1,
    horizontal_spacing=0.125,
    color_data=REACTIVE_ZONES_TOT[project],
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            4: "Typically in heating",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            4: RESPONSE_COLORS["Typically in heating"],
        },
    },
    y_zerolinecolor="LightGray",
    legend_size=MULTI_PLOT_TXT_SIZE + 10,
    text_size=MULTI_PLOT_TXT_SIZE + 6,
    width=1700,
    height=550,
    marker_size=14,
)

draw_dominant_y = {}
draw_dominant_x = {}
# detemrine lines

# rogue
these_rogue = list(
    DOMINANT_ZONES_TOT[project][DOMINANT_ZONES_TOT[project] == 2].dropna().index
)
this_lorenz_rogue = LORENZ_ALL_TOT[project].loc[these_rogue, :].iloc[:, 0]
this_lorenz_rogue = this_lorenz_rogue.sort_values(ascending=False)

# dominant
these_dominant = list(
    DOMINANT_ZONES_TOT[project][DOMINANT_ZONES_TOT[project] == 1].dropna().index
)
this_lorenz_dominant = LORENZ_ALL_TOT[project].loc[these_dominant, :].iloc[:, 0]
this_lorenz_dominant = this_lorenz_dominant.sort_values(ascending=False)
try:
    this_dominant = float(this_lorenz_dominant.iloc[0])
except:
    this_dominant = np.nan
draw_dominant_y[project] = this_dominant
draw_dominant_x[project] = (
    len(this_lorenz_dominant) + len(this_lorenz_rogue) - 1
) / len(LORENZ_ALL_TOT[project])

i = 1
xref = "x" if i == 1 else f"x{i}"
yref = "y" if i == 1 else f"y{i}"

fig.add_shape(
    type="line",
    x0=-0.1,
    x1=draw_dominant_x[project],
    y0=draw_dominant_y[project],
    y1=draw_dominant_y[project],
    line=dict(color="black", dash="dash", width=3),
    xref=xref,
    yref=yref,
    layer="above",
)
fig.add_shape(
    type="line",
    x0=draw_dominant_x[project],
    x1=draw_dominant_x[project],
    y0=-0.1,
    y1=draw_dominant_y[project],
    line=dict(color="black", dash="dash", width=3),
    xref=xref,
    yref=yref,
    layer="above",
)
fig.add_annotation(
    text="Dominant",
    x=draw_dominant_x[project],
    y=draw_dominant_y[project],
    xanchor="left",
    yanchor="top",
    showarrow=False,
    font=dict(size=MULTI_PLOT_ANNOTATION_SIZE + 14, color="black"),
    xref=xref,
    yref=yref,
    textangle=90,
)

fig = fig.update_yaxes(
    tickmode="array",
    tickvals=[-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1.0],
    ticktext=["", "0", "0.2", "0.4", "0.6", "0.8", "1.0"],
    matches="y",  # link all y-axes
)

In [ ]:
fig.write_image(f"{IMAGE_PATH}/GraphAbstractTopRight.png")

In [ ]:
# fig

### For presentation

In [ ]:
dzs_tot = get_dominant_zones(
    LORENZ_ALL_TOT,
    CRs_CONTROL_MEAN_TOT,
    dominant_thresh=DOMINANT_THRESH,
    rogue_thresh=ROGUE_THRESH,
    dominant_greater_lesser="lesser",
    to_csv=True,
)
dzs_tot = dzs_tot["TOTAL"]

In [ ]:
project = "TOTAL"

fig = viz.make_dot_plot(
    LORENZ_ALL_TOT[project],
    y_range=[-0.1, 1.1],
    x_range=[-0.1, 1.1],
    normalize_x=True,
    y_axis_title="Fraction of Cumulative<br>Cooling Requests [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.1,
    horizontal_spacing=0.125,
    color_data=REACTIVE_ZONES_TOT[project],
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: "Other",
            4: "Other",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: "Gray",
            4: "Gray",
        },
    },
    y_zerolinecolor="LightGray",
    legend_size=MULTI_PLOT_TXT_SIZE + 10,
    text_size=MULTI_PLOT_TXT_SIZE + 6,
    width=1000,
    height=800,
    marker_size=14,
    dont_add_to_legend=[4],
)

fig = fig.update_yaxes(
    tickmode="array",
    tickvals=[-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1.0],
    ticktext=["", "0", "0.2", "0.4", "0.6", "0.8", "1.0"],
    matches="y",  # link all y-axes
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.25,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
# detemrine lines
draw_dominant_y = {}
draw_dominant_x = {}
draw_rogue_y = {}
draw_rogue_x = {}

# rogue
these_rogue = list(dzs_tot[dzs_tot == 2].dropna().index)
this_lorenz_rogue = LORENZ_ALL_TOT[project].loc[these_rogue, :].iloc[:, 0]
this_lorenz_rogue = this_lorenz_rogue.sort_values(ascending=False)
this_rogue = float(this_lorenz_rogue.iloc[0])
draw_rogue_y[project] = this_rogue
draw_rogue_x[project] = (len(this_lorenz_rogue) - 1) / len(LORENZ_ALL_TOT[project])

# dominant
these_dominant = list(dzs_tot[dzs_tot == 1].dropna().index)
this_lorenz_dominant = LORENZ_ALL_TOT[project].loc[these_dominant, :].iloc[:, 0]
this_lorenz_dominant = this_lorenz_dominant.sort_values(ascending=False)
this_dominant = float(this_lorenz_dominant.iloc[0])
draw_dominant_y[project] = this_dominant
draw_dominant_x[project] = (
    len(this_lorenz_dominant) + len(this_lorenz_rogue) - 1
) / len(LORENZ_ALL_TOT[project])

i = 1
xref = "x" if i == 1 else f"x{i}"
yref = "y" if i == 1 else f"y{i}"

# Rogue
fig = fig.add_shape(
    type="line",
    x0=-0.1,
    x1=draw_rogue_x[project],
    y0=draw_rogue_y[project],
    y1=draw_rogue_y[project],
    line=dict(color="black", dash="dash", width=3),
    xref=xref,
    yref=yref,
    layer="above",
)
fig = fig.add_shape(
    type="line",
    x0=draw_rogue_x[project],
    x1=draw_rogue_x[project],
    y0=-0.1,
    y1=draw_rogue_y[project],
    line=dict(color="black", dash="dash", width=3),
    xref=xref,
    yref=yref,
    layer="above",
)
fig = fig.add_annotation(
    text="Rogue",
    x=draw_rogue_x[project],
    y=draw_rogue_y[project],
    xanchor="left",
    yanchor="top",
    showarrow=False,
    font=dict(size=MULTI_PLOT_ANNOTATION_SIZE + 14, color="black"),
    xref=xref,
    yref=yref,
    textangle=90,
)

# Dominant
fig = fig.add_shape(
    type="line",
    x0=-0.1,
    x1=draw_dominant_x[project],
    y0=draw_dominant_y[project],
    y1=draw_dominant_y[project],
    line=dict(color="black", dash="dash", width=3),
    xref=xref,
    yref=yref,
    layer="above",
)
fig = fig.add_shape(
    type="line",
    x0=draw_dominant_x[project],
    x1=draw_dominant_x[project],
    y0=-0.1,
    y1=draw_dominant_y[project],
    line=dict(color="black", dash="dash", width=3),
    xref=xref,
    yref=yref,
    layer="above",
)
fig = fig.add_annotation(
    text="Dominant",
    x=draw_dominant_x[project],
    y=draw_dominant_y[project],
    xanchor="left",
    yanchor="top",
    showarrow=False,
    font=dict(size=MULTI_PLOT_ANNOTATION_SIZE + 14, color="black"),
    xref=xref,
    yref=yref,
    textangle=90,
)

In [ ]:
# fig

## Heat map 

In [ ]:
(
    heat_maps_row,
    heat_maps_col,
) = quantify_heatmap_dfs(DOMINANT_ZONES_TOT, REACTIVE_ZONES_TOT)

In [ ]:
fig = plot_confusion_heatmaps(
    heat_maps_row["TOTAL"],
    title="Row Normalized",
    text_size=SINGLE_PLOT_TXT_SIZE,
    width=1200,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC9.png")

In [ ]:
fig = plot_confusion_heatmaps(
    heat_maps_col["TOTAL"],
    title="Column Normalized",
    text_size=SINGLE_PLOT_TXT_SIZE,
    width=1200,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC10.png")

# Consider alternative definitions 

## Zone level cooling

In [ ]:
(cooling, cooling_control, cooling_control_mean) = get_data(
    this_var="zone-total_cooling",
    projects=PROJECTS_VAV,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=True,
    clean_underyling_data=True,
    clean_this_var="zone-dummy",
)
cooling_control_mean_tot = add_total_to_results_dict(cooling_control_mean)

In [ ]:
lorenz_all_alt1_tot = get_lorenz_curves(cooling_control_mean_tot)

In [ ]:
dominant_zones_alt1_tot = get_dominant_zones(
    lorenz_all_alt1_tot,
    CRs_CONTROL_MEAN_TOT,
    dominant_thresh=DOMINANT_THRESH,
    rogue_thresh=ROGUE_THRESH,
    dominant_greater_lesser="lesser",
    to_csv=False,
)

In [ ]:
count_norm_alt1_tot = count_dzs(dominant_zones_alt1_tot)

In [ ]:
# 100 * count_norm_alt1_tot

In [ ]:
fig = viz.make_dot_plot(
    lorenz_all_alt1_tot,
    normalize_x=True,
    y_range=[-0.1, 1.1],
    y_axis_title="Fraction of<br>Cumulative Cooling<br>Control Days [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.125,
    horizontal_spacing=0.125,
    color_data=REACTIVE_ZONES_TOT,
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            4: "Typically in heating",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            4: RESPONSE_COLORS["Typically in heating"],
        },
    },
    y_zerolinecolor="LightGray",
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.075,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)
fig = add_gini(fig, lorenz_all_alt1_tot, cooling_control_mean_tot)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD1.png")

## Airflow

In [ ]:
(airflow, airflow_control, airflow_control_mean) = get_data(
    this_var="zone-airflow",
    projects=PROJECTS_VAV,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=True,
    clean_this_var="zone-airflow",
)

In [ ]:
airflow_control_mean_tot = add_total_to_results_dict(airflow_control_mean)

In [ ]:
lorenz_all_alt2_tot = get_lorenz_curves(airflow_control_mean_tot)

In [ ]:
dominant_zones_alt2_tot = get_dominant_zones(
    lorenz_all_alt2_tot,
    CRs_CONTROL_MEAN_TOT,
    dominant_thresh=DOMINANT_THRESH,
    rogue_thresh=ROGUE_THRESH,
    dominant_greater_lesser="lesser",
    to_csv=False,
)

In [ ]:
count_norm_alt2_tot = count_dzs(dominant_zones_alt2_tot)

In [ ]:
# 100 * count_norm_alt2_tot

In [ ]:
fig = viz.make_dot_plot(
    lorenz_all_alt2_tot,
    y_range=[-0.1, 1.1],
    normalize_x=True,
    y_axis_title="Fraction of<br>Cumulative Airflow<br>Control Days [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.125,
    horizontal_spacing=0.125,
    color_data=REACTIVE_ZONES_TOT,
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            4: "Typically in heating",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            4: RESPONSE_COLORS["Typically in heating"],
        },
    },
    marker_size=10,
    y_zerolinecolor="LightGray",
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.075,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)
fig = add_gini(fig, lorenz_all_alt2_tot, airflow_control_mean_tot)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD2.png")

## CRs

In [ ]:
dominant_zones_alt3_tot = get_dominant_zones(
    CRs_CONTROL_MEAN_TOT,
    CRs_CONTROL_MEAN_TOT,
    0.1,
    rogue_thresh=ROGUE_THRESH,
    dominant_greater_lesser="greater",
    to_csv=False,
)

In [ ]:
crs_control_std = base.run_passive_test_on_dfs(
    dfs=crs_control, this_test="Std", col_name="Std"
)
crs_control_std_tot = add_total_to_results_dict(crs_control_std)

In [ ]:
fig = viz.make_dot_plot(
    y_data=CRs_CONTROL_MEAN_TOT,
    # y_error_up_data=crs_control_std_tot,
    # y_error_down_data=crs_control_std_tot,
    y_axis_title="Fraction of Time<br>Sending Cooling Requests<br>Control Days [Unitless]",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.1,
    horizontal_spacing=0.125,
    y_range=[0, 1],
    color_data=REACTIVE_ZONES_TOT,
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            4: "Typically in heating",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            4: RESPONSE_COLORS["Typically in heating"],
        },
    },
    normalize_x=True,
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)
fig = add_line_to_subplots(
    fig=fig,
    x_range=[-0.1, 1.1],
    y_range=[0.1, 0.1],
    total_subplots=len(CRs_CONTROL_MEAN_TOT.keys()),
    dash="solid",
    width=3,
)

fig = add_line_to_subplots(
    fig=fig,
    x_range=[-0.1, 1.1],
    y_range=[0.7, 0.7],
    total_subplots=len(CRs_CONTROL_MEAN_TOT.keys()),
    dash="solid",
    width=3,
)

for i in range(1, len(CRs_CONTROL_MEAN_TOT.keys()) + 1):
    project = list(CRs_CONTROL_MEAN_TOT.keys())[i - 1]
    xref = "x" if i == 1 else f"x{i}"
    yref = "y" if i == 1 else f"y{i}"
    fig.add_annotation(
        text="Rogue",
        x=0,
        y=0.85,
        xanchor="left",
        yanchor="middle",
        showarrow=False,
        font=dict(size=MULTI_PLOT_ANNOTATION_SIZE, color="black"),
        xref=xref,
        yref=yref,
    )
    fig.add_annotation(
        text="Dominant",
        x=0,
        y=0.4,
        xanchor="left",
        yanchor="middle",
        showarrow=False,
        font=dict(size=MULTI_PLOT_ANNOTATION_SIZE, color="black"),
        xref=xref,
        yref=yref,
    )

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD3.png")

In [ ]:
count_norm_alt3_tot = count_dzs(dominant_zones_alt3_tot)

In [ ]:
# 100 * (count_norm_alt3_tot['Rogue'] + count_norm_alt3_tot['Dominant'])

In [ ]:
fig = viz.make_bar_plot(
    y_data=count_norm_alt3_tot,
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        }
    },
    y_axis_title="Fraction of Zones [Unitless]",
    y_range=[0, 1],
    annotations=[
        "Dominated",
        "Dominant",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(count_norm_alt3_tot.index))],
    width=1200,
    height=600,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    xaxis=dict(range=[-0.25, len(count_norm_alt3_tot.index)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD4.png")

In [ ]:
count_CRs_norm_alt3_tot = count_dz_crs(dominant_zones_alt3_tot, CRs_CONTROL_MEAN_TOT)

In [ ]:
# 100 * (count_CRs_norm_alt3_tot['Rogue'] + count_CRs_norm_alt3_tot['Dominant'])

In [ ]:
fig = viz.make_bar_plot(
    y_data=count_CRs_norm_alt3_tot,
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        }
    },
    y_axis_title="Fraction of Cooling Requests<br>Control Days [Unitless]",
    y_range=[0, 1],
    annotations=[
        "Dominated",
        "Dominant",
        "Rogue",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(count_CRs_norm_alt3_tot.index))],
    width=1200,
    height=600,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    xaxis=dict(range=[-0.25, len(count_CRs_norm_alt3_tot.index)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD5.png")

In [ ]:
(
    heat_maps_row_alt3_tot,
    heat_maps_col_alt3_tot,
) = quantify_heatmap_dfs(dominant_zones_alt3_tot, REACTIVE_ZONES_TOT)

In [ ]:
fig = plot_confusion_heatmaps(
    heat_maps_row_alt3_tot["TOTAL"],
    title="Row Normalized",
    text_size=SINGLE_PLOT_TXT_SIZE,
    width=1200,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD6.png")

In [ ]:
fig = plot_confusion_heatmaps(
    heat_maps_col_alt3_tot["TOTAL"],
    title="Column Normalized",
    text_size=SINGLE_PLOT_TXT_SIZE,
    width=1200,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD7.png")

## Tloads

In [ ]:
dominant_zones_alt4_tot = get_dominant_zones(
    TLOADS_CONTROL_MEAN_TOT,
    CRs_CONTROL_MEAN_TOT,
    dominant_thresh=20,
    rogue_thresh=ROGUE_THRESH,
    dominant_greater_lesser="greater",
    to_csv=False,
)

In [ ]:
tloads_control_std = base.run_passive_test_on_dfs(
    dfs=tloads_control, this_test="Std", col_name="Std"
)
tloads_control_std_tot = add_total_to_results_dict(tloads_control_std)

In [ ]:
fig = viz.make_dot_plot(
    y_data=TLOADS_CONTROL_MEAN_TOT,
    # y_error_up_data=tloads_control_std_tot,
    # y_error_down_data=tloads_control_std_tot,
    y_axis_title="Zonal Load [%]<br>Control Days",
    x_axis_title="Fraction of Zones [Unitless]",
    vertical_spacing=0.1,
    horizontal_spacing=0.1,
    y_range=[0, 100],
    color_data=REACTIVE_ZONES_TOT,
    color_legend={
        "name": {
            0: f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more",
            1: "Small change zonal load",
            2: "Small change zonal load (remained high)",
            3: f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more",
            4: "Typically in heating",
        },
        "color": {
            0: RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            1: RESPONSE_COLORS["Small change zonal load"],
            2: RESPONSE_COLORS["Small change zonal load (remained high)"],
            3: RESPONSE_COLORS[
                f"Increased zonal load {abs(REACTIVE_THRESH['Pos'])}% or more"
            ],
            4: RESPONSE_COLORS["Typically in heating"],
        },
    },
    normalize_x=True,
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)
fig = add_line_to_subplots(
    fig=fig,
    x_range=[-0.1, 1.1],
    y_range=[20, 20],
    total_subplots=len(TLOADS_CONTROL_MEAN_TOT.keys()),
    dash="solid",
    width=3,
)

draw_rogue_y = {}

# detemrine lines
for project in LORENZ_ALL_TOT:
    # rogue
    these_rogue = list(
        dominant_zones_alt4_tot[project][dominant_zones_alt4_tot[project] == 2]
        .dropna()
        .index
    )
    this_tloads_rogue = TLOADS_CONTROL_MEAN_TOT[project].loc[these_rogue, :].iloc[:, 0]
    this_tloads_rogue = this_tloads_rogue.sort_values(ascending=True)
    try:
        this_rogue = float(this_tloads_rogue.iloc[0])
    except:
        this_rogue = np.nan
    draw_rogue_y[project] = this_rogue

for i in range(1, len(CRs_CONTROL_MEAN_TOT.keys()) + 1):
    project = list(CRs_CONTROL_MEAN_TOT.keys())[i - 1]
    xref = "x" if i == 1 else f"x{i}"
    yref = "y" if i == 1 else f"y{i}"
    if not np.isnan(draw_rogue_y[project]):
        fig.add_shape(
            type="line",
            x0=-0.1,
            x1=1.1,
            y0=draw_rogue_y[project],
            y1=draw_rogue_y[project],
            line=dict(color="black", dash="solid", width=3),
            xref=xref,
            yref=yref,
            layer="above",
        )
        fig.add_annotation(
            text="Rogue",
            x=0,
            y=(100 + draw_rogue_y[project]) / 2,
            xanchor="left",
            yanchor="middle",
            showarrow=False,
            font=dict(size=MULTI_PLOT_ANNOTATION_SIZE, color="black"),
            xref=xref,
            yref=yref,
        )
    fig.add_annotation(
        text="Dominant",
        x=0,
        y=(draw_rogue_y[project] + 20) / 2,
        xanchor="left",
        yanchor="middle",
        showarrow=False,
        font=dict(size=MULTI_PLOT_ANNOTATION_SIZE, color="black"),
        xref=xref,
        yref=yref,
    )

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD8.png")

In [ ]:
count_norm_alt4_tot = count_dzs(dominant_zones_alt4_tot)

In [ ]:
# 100 * count_norm_alt4_tot

In [ ]:
# 100 * (count_norm_alt4_tot["Rogue"] + count_norm_alt4_tot["Dominant"])

In [ ]:
fig = viz.make_bar_plot(
    y_data=count_norm_alt4_tot,
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        }
    },
    y_axis_title="Fraction of Zones [Unitless]",
    y_range=[0, 1],
    annotations=[
        "Dominated",
        "Dominant",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(count_norm_alt4_tot.index))],
    width=1200,
    height=600,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    xaxis=dict(range=[-0.25, len(count_norm_alt4_tot.index)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD9.png")

In [ ]:
count_CRs_norm_alt4_tot = count_dz_crs(dominant_zones_alt4_tot, CRs_CONTROL_MEAN_TOT)

In [ ]:
# 100 * (count_CRs_norm_alt4_tot['Rogue'] + count_CRs_norm_alt4_tot['Dominant'])

In [ ]:
fig = viz.make_bar_plot(
    y_data=count_CRs_norm_alt4_tot,
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        }
    },
    y_axis_title="Fraction of Cooling Requests<br>Control Days [Unitless]",
    y_range=[0, 1],
    annotations=[
        "Dominated",
        "Dominant",
        "Rogue",
    ],
    annotation_thresh=0.05,
    bar_width=0.75,
    bar_mode="stack",
    tick_vals=[i + 0.5 for i in range(len(count_CRs_norm_alt4_tot.index))],
    width=1200,
    height=600,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
    xaxis=dict(range=[-0.25, len(count_CRs_norm_alt4_tot.index)]),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD10.png")

In [ ]:
(heat_map_row_alt4_tot, heat_map_col_alt4_tot) = quantify_heatmap_dfs(
    dominant_zones_alt4_tot, REACTIVE_ZONES_TOT
)

In [ ]:
fig = plot_confusion_heatmaps(
    heat_map_row_alt4_tot["TOTAL"],
    title="Row Normalized",
    text_size=SINGLE_PLOT_TXT_SIZE,
    width=1200,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD11.png")

In [ ]:
fig = plot_confusion_heatmaps(
    heat_map_col_alt4_tot["TOTAL"],
    title="Column Normalized",
    text_size=SINGLE_PLOT_TXT_SIZE,
    width=1200,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureD12.png")